# SKA SDP AA0.5LOW Emulated Receive Workflow

Instructions for how to test the persistent sdp receive using emulated AA0.5LOW data with tango and sdp interfaces. This notebook can be executed remotely using binderhub via the following link:

https://sdhp.stfc.skao.int/binderhub/v2/gl/ska-telescope%2Fsdp%2Fska-sdp-notebooks/HEAD

## Tango Proxy Interface Intro

The Tango device proxy interface provides interaction to a subarray and it's associated execution block in the form of a state machine. When the device is On, this interface provides a single observable state object:

#### ObsState

* EMPTY
* IDLE
* READY
* SCANNING

Communication to the Tango device is performed via the use of commands and accessors whereby all data is conformant to the sdp schemas available here:

https://developer.skao.int/projects/ska-telmodel/en/latest/schemas/ska-sdp.html


| Command    | Current State | Next State | Input  | Output |
| ---------- | ------------- | ---------- | ------ | ------ |
| AssignRes  |     EMPTY     |   IDLE     | https://developer.skao.int/projects/ska-telmodel/en/latest/schemas/ska-sdp-assignres.html | None
| Configure  |     IDLE      |   READY    | https://developer.skao.int/projects/ska-telmodel/en/latest/schemas/ska-sdp-configure.html | None
| Scan       |     READY     |  SCANNING  | https://developer.skao.int/projects/ska-telmodel/en/latest/schemas/ska-sdp-scan.html | None
| RecvAddrs  |    SCANNING   |  SCANNING  | None | https://developer.skao.int/projects/ska-telmodel/en/latest/schemas/ska-sdp-recvaddrs.html
| EndScan    |    SCANNING   |   READY    | None | None
| End        |     READY     |   IDLE     | None | None
| ReleaseRes |     IDLE      |   EMPTY    | None | None

Note: The next state transition is not instantaneous and should be waited for before executing another command.

### Execution Block

A tango device monitors exactly 1 execution block that is defined by the configure command.

### Processing Block

Individual processes running in an execution block when not in the EMPTY state.

# Install Kubernetes (Dev)

In [ ]:
# Debug using python kubernetes
!pip install kubernetes

# Alternatively debug using kubectl commands
!curl -LO "https://dl.k8s.io/release/$(curl -L -s https://dl.k8s.io/release/stable.txt)/bin/linux/amd64/kubectl"
!sudo install -o root -g root -m 0755 kubectl /usr/local/bin/kubectl
!kubectl --help

# In addition to this step, you must provide a kubeconfig file which is by default located at:
# $HOME/.kube/config
# or KUBECONFIG environment variable 
# or can be passed to load_kube_config()

# Install Helm (Dev)

In [ ]:
!sudo apt install -y gpg
!curl https://baltocdn.com/helm/signing.asc | gpg --dearmor | sudo tee /usr/share/keyrings/helm.gpg > /dev/null
!sudo apt-get install apt-transport-https --yes
!echo "deb [arch=$(dpkg --print-architecture) signed-by=/usr/share/keyrings/helm.gpg] https://baltocdn.com/helm/stable/debian/ all main" | sudo tee /etc/apt/sources.list.d/helm-stable-debian.list
!sudo apt-get update
!sudo apt-get install helm
!helm --help

# Connect to SDP

In [304]:
from tango import DeviceProxy, EventType
import ska_sdp_config
import os
import json
import random
from datetime import date

# kubernetes client requires a kubeconfig and is only available for development 
debug = True
if debug:
    import kubernetes
    KUBECONFIG = "/app/config" # or "$HOME/.kube/config"
    k8s_client = kubernetes.client.api_client.ApiClient(configuration=kubernetes.config.load_kube_config(kubeconfig))
    v1 = kubernetes.client.CoreV1Api()

shared = False
if shared:
    # Make sure you connect to the correct Configuration Database
    KUBE_NAMESPACE = "dp-shared"
    KUBE_PROC_NAMESPACE = f"{KUBE_NAMESPACE}-p"
else:
    KUBE_NAMESPACE = "dp-yanda-callan"  # add the namespace you want to connect to here
    KUBE_PROC_NAMESPACE = f"{KUBE_NAMESPACE}-p"
    # install the sdp
    !helm repo add ska https://artefact.skao.int/repository/helm-internal
    !KUBECONFIG={KUBECONFIG} helm uninstall test-sdp --namespace {KUBE_NAMESPACE} --wait
    !KUBECONFIG={KUBECONFIG} helm upgrade --install test-sdp ska/ska-sdp --namespace {KUBE_NAMESPACE} --set helmdeploy.namespace={KUBE_PROC_NAMESPACE} --wait
    # Alternative sdp install from local repo
    #!git clone https://gitlab.com/ska-telescope/sdp/ska-sdp-integration.git --init --recursive
    #!KUBECONFIG={KUBECONFIG} make uninstall-sdp -C ./ska-sdp-integration KUBE_NAMESPACE={KUBE_NAMESPACE} KUBE_NAMESPACE_SDP={KUBE_PROC_NAMESPACE} --wait
    #!KUBECONFIG="/app/config" make install-sdp -C ./ska-sdp-integration KUBE_NAMESPACE={KUBE_NAMESPACE} KUBE_NAMESPACE_SDP=dp-yanda-callan-p
    
os.environ["SDP_CONFIG_HOST"] = f"ska-sdp-etcd-client.{KUBE_NAMESPACE}"

# set the name of the databaseds service
DATABASEDS_NAME = "databaseds-tango-base"

# finally set the TANGO_HOST
os.environ["TANGO_HOST"] = f"{DATABASEDS_NAME}.{KUBE_NAMESPACE}.svc.cluster.local:10000"
d = DeviceProxy('test-sdp/subarray/02')
d.set_logging_level(5)
config = ska_sdp_config.Config()


"ska" already exists with the same configuration, skipping
release "persistent-sdp" uninstalled
Release "persistent-sdp" does not exist. Installing it now.
NAME: persistent-sdp
LAST DEPLOYED: Fri Sep 16 07:55:04 2022
NAMESPACE: dp-yanda-callan
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
SKA SDP deployed!

You can connect to the configuration database by running a shell in the
console pod. To start a bash shell, use:

    $ kubectl -n dp-yanda-callan exec -it ska-sdp-console-0 -- bash

and from there you can use the ska-sdp command, e.g.:

    # ska-sdp list -a


In [305]:
print(KUBE_PROC_NAMESPACE)
print(os.environ["SDP_CONFIG_HOST"])
print(os.environ["TANGO_HOST"])
d = DeviceProxy('test-sdp/subarray/01')

dp-yanda-callan-p
ska-sdp-etcd-client.dp-yanda-callan
databaseds-tango-base.dp-yanda-callan.svc.cluster.local:10000


In [298]:
# Check SDP deployment
!KUBECONFIG={KUBECONFIG} helm list -n {KUBE_NAMESPACE}
!KUBECONFIG={KUBECONFIG} kubectl get pods -n {KUBE_NAMESPACE}
!KUBECONFIG={KUBECONFIG} kubectl get pods -n {KUBE_PROC_NAMESPACE}

NAME          	NAMESPACE      	REVISION	UPDATED                                	STATUS  	CHART         	APP VERSION
persistent-sdp	dp-yanda-callan	5       	2022-09-16 07:49:10.700466971 +0000 UTC	deployed	ska-sdp-0.11.2	0.11.2     
NAME                              READY   STATUS      RESTARTS        AGE
databaseds-tango-base-0           1/1     Running     1 (3h39m ago)   3h39m
ska-sdp-console-0                 1/1     Running     0               3h39m
ska-sdp-etcd-0                    1/1     Running     0               3h39m
ska-sdp-helmdeploy-0              1/1     Running     0               3h39m
ska-sdp-lmc-config--1-9k6tk       0/1     Completed   0               3h39m
ska-sdp-lmc-controller-0          1/1     Running     0               3h39m
ska-sdp-lmc-subarray-01-0         1/1     Running     0               3h39m
ska-sdp-opinterface-0             1/1     Running     0               3h39m
ska-sdp-proccontrol-0             1/1     Running     0               3h39m
ska-sdp-sc

In [306]:
# Utilities

import pytest
import time

TIMEOUT = 60.0  # seconds
INTERVAL = 0.5  # seconds

def wait_for_predicate(
    predicate, description, timeout=TIMEOUT, interval=INTERVAL
):
    """
    Wait for predicate to be true.

    :param predicate: callable to test
    :param description: description to use if test fails
    :param timeout: timeout in seconds
    :param interval: interval between tests of the predicate in seconds

    """
    start = time.time()
    while True:
        if predicate():
            break
        if time.time() >= start + timeout:
            pytest.fail(f"{description} not achieved after {timeout} seconds")
        time.sleep(interval)


def wait_for_state(device, state, timeout=TIMEOUT):
    """
    Wait for device state to have the expected value.

    :param device: device client
    :param state: the expected state
    :param timeout: timeout in seconds

    """

    def predicate():
        return device.state() == state

    description = f"Device state {state.name}"
    print(f"Waiting for device state {state.name}...")
    wait_for_predicate(predicate, description, timeout=timeout)


def wait_for_obs_state(device, obs_state, timeout=TIMEOUT):
    """
    Wait for obsState to have the expected value.

    :param device: device proxy
    :param obs_state: the expected value
    :param timeout: timeout in seconds
    """

    def predicate():
        return device.obsState == obs_state

    description = f"obsState {obs_state.name}"
    print(f"Waiting for device obs_state {obs_state.name}...")
    wait_for_predicate(predicate, description, timeout=timeout)

def tango_safe_release():
    """
    Safely releases tango device to EMPTY obsState
    """
    if d.obsState == d.obsState.SCANNING:
        print(">> End Scan")
        d.EndScan()
        wait_for_obs_state(d, d.obsState.READY)

    if d.obsState == d.obsState.READY:
        print(">> End")
        d.End()
        wait_for_obs_state(d, d.obsState.IDLE)

    if d.obsState == d.obsState.IDLE:
        print(">> Releasing Resources")
        d.ReleaseResources()        
        wait_for_obs_state(d, d.obsState.EMPTY)
        
    # if state FAULT:
    # d.Restart()
    
    assert d.obsState == d.obsState.EMPTY
    print("Tango Device is EMPTY")
    
def tango_safe_off():
    """
    Safely turns tango device to OFF state
    """
    tango_safe_release()
    if d.state() == DevState.ON:
        print(">> Device OFF")
        d.Off()
        wait_for_state(d, DevState.OFF)

    assert d.state() == DevState.OFF
    print("Tango Device is OFF")
    
#d.Restart()
tango_safe_off()

>> End Scan
Waiting for device obs_state READY...
>> End
Waiting for device obs_state IDLE...
>> Releasing Resources
Waiting for device obs_state EMPTY...
Tango Device is EMPTY
>> Device OFF
Waiting for device state OFF...
Tango Device is OFF


# Configure Processing Block Parameters

Before creating an execution block, parameters may need to be passed to the workflow script in ska-sdp-scripts to ensure correct reception is configured correctly.

In [121]:
def create_receive_parameters():
    model_ms            = "AA05LOW.ms"
    output_ms           = "AA05LOW.ms"
    total_channels      = 13824
    total_timesteps     = 6
    total_streams       = 4
    num_repeats         = 3
    rate                = 10416667  # bits per second
    channels_per_stream = total_channels // total_streams
    max_payloads        = total_timesteps * total_streams * num_repeats
    max_payload_misses  = 30  # payload timeout in seconds
    max_ms              = 1  # -1 to continuously run

    return {
      "image": "artefact.skao.int/ska-sdp-realtime-receive-modules",
      "version": "3.2.1",
      "model": {
        "name": model_ms
      },
      "transmission": {
        "transport_protocol": "udp",
        "channels_per_stream": channels_per_stream,
        "rate": str(rate)
      },
      "payload.method": "icd",
      "reader": {
        "num_timestamps": 0,
        "start_chan": 0,
        "num_chan": 0,
        "num_repeats": num_repeats,
      },
      "reception": {
        "outputfilename": "/mnt/output.ms",
        "receiver_port_start": "41000",
        "num_ports": str(total_streams),
        "ring_heaps": 133,
        "consumer": "plasma_writer",
        "use_plasm_ms": False,
        "plasma_path": "/plasma/socket",
      },
      "pvc": {
          "name": "shared",
          "path": "/mnt/data",
      },
      "extraVolumeMounts": [
        {
            "name": "plasma-storage-volume",
            "mountPath": "/plasma",
        }
      ],
      "extraVolumes": [
        {
          "name": "plasma-storage-volume",
          "emptyDir": {
            "medium": "Memory"
          }
        }
      ],
      "extraContainers": [
        {
          "name": "plasma-store",
         "image": "artefact.skao.int/ska-sdp-realtime-receive-modules",
         "command": ["plasma_store", "-s", "/plasma/socket", "-m", "1000000000"],
          "volumeMounts": [
            {
             "name": "plasma-storage-volume",
              "mountPath": "/plasma"
            },
          ]
        },
        {
          "name": "plasma-processor",
          "image": "artefact.skao.int/ska-sdp-realtime-receive-modules",
          "version": "3.2.1",
          "command": ["sh", "-c", " ".join([
                "plasma-mswriter",
                "-s", "/plasma/socket",
                "--max_payloads", str(max_payloads),
                "--max_ms", str(max_ms),
                "--max_payload_misses", str(max_payload_misses),
                "--use_plasma_ms", "False",
                "--ms_command", f"'mv %s /mnt/data/{output_ms}'",
                "/mnt/output.ms",
          ])],
          "volumeMounts": [
            {
              "name": "plasma-storage-volume",
              "mountPath": "/plasma"
            },
            { # for writing to output directory
              "name": "local-pvc",
              "mountPath": "/mnt/data"
            }
          ]
        }
      ],
    }


def create_sender_parameters(receive_address: str, receive_port: str):
    input_ms            = "AA05LOW.ms"
    total_channels      = 13824
    total_timesteps     = 6
    total_streams       = 4
    num_repeats         = 3
    rate                = 10416667  # bits per second
    channels_per_stream = total_channels // total_streams
    return \
    {
      "apiVersion": "batch/v1",
      "kind": "Job",
      "metadata": {
        "labels": {
          "app.kubernetes.io/name": "sendtest"
        },
        "name": "sendtest",
        "namespace": "sdp"
      },
      "spec": { "template": { "spec": {
        "volumes": [
          {
            "name": "local-pvc",
            "persistentVolumeClaim": {
              "claimName": "local-pvc"
            }
          }
        ],
        "containers": [
          {
            "command": ["/bin/bash", "-c", " ".join([
              'emu-send',
              f'-o transmission.method=spead2_transmitters',
              f'-o transmission.channels_per_stream={channels_per_stream}',
              f'-o transmission.rate={rate}',
              f'-o transmission.target_host={receive_address}',
              f'-o transmission.target_start_port={receive_port}',
              f'-o reader.num_repeats={num_repeats}',
              f'/app/{input_ms}'
            ])],
            "image": "artefact.skao.int/ska-sdp-cbf-emulator",
            "version": "3.1.0",
            "imagePullPolicy": "IfNotPresent",
            "name": "sendtest",
            "resources": {},
            "terminationMessagePath": "/dev/termination-log",
            "terminationMessagePolicy": "File",
            "volumeMounts": [
              {
                "mountPath": "/mnt/data",
                "name": "local-pvc"
              }
            ]
          }
        ],
        "restartPolicy": "Never"
      }}}
    }

# Configure Execution Block

In [300]:
# list of available workflows
!ska-sdp list script

Keys with prefix /script: 
/script/batch:test-batch:0.3.0
/script/batch:test-daliuge:0.3.0
/script/batch:test-dask:0.3.0
/script/realtime:pss-receive:0.3.0
/script/realtime:test-realtime:0.3.0
/script/realtime:test-receive-addresses:0.4.0
/script/realtime:vis-receive:0.5.0
/script/realtime:vis-receive:0.5.1


In [307]:
def create_resources_config():
    generator = "notebook"
    today = date.today().strftime("%Y%m%d")
    number = random.randint(0, 99998)

    EXECUTION_BLOCK_ID = f"eb-{generator}-{today}-{number:05d}"
    PROCESSING_BLOCK_ID_REALTIME_SENDER = f"pb-{generator}-{today}-{number:05d}"
    PROCESSING_BLOCK_ID_REALTIME_RECEIVER = f"pb-{generator}-{today}-{number+1:05d}"
    PROCESSING_BLOCK_ID_BATCH = f"pb-{generator}-{today}-{number+2:05d}"

    return {
      "interface": "https://schema.skao.int/ska-sdp-assignres/0.3",
      "eb_id":f"{EXECUTION_BLOCK_ID}",
      "max_length": 21600.0,
      "scan_types": [
        {
          "scan_type_id": "science",
          "coordinate_system": "ICRS", "ra": "02:42:40.771", "dec": "-00:00:47.84",
          "channels": [
            { "count": 10, "start": 0, "stride": 2, "freq_min": 0.35e9, "freq_max": 0.368e9, "link_map": [[0,0], [200,1], [744,2], [944,3]] },
          ]
        },
        {
          "scan_type_id": "calibration",
          "coordinate_system": "ICRS", "ra": "12:29:06.699", "dec": "02:03:08.598",
          "channels": [
            { "count": 5, "start": 0, "stride": 2, "freq_min": 0.35e9, "freq_max": 0.368e9, "link_map": [[0,0], [200,1], [744,2], [944,3]] },
          ]
        }
      ],
      "processing_blocks": [
        {
          "pb_id": f"{PROCESSING_BLOCK_ID_REALTIME_RECEIVER}",
          #"workflow": {"kind": "realtime", "name": "test-receive-addresses", "version": "0.4.0"},
          "workflow": {"kind": "realtime", "name": "vis-receive", "version": "0.5.1"},
          #"parameters": create_receive_parameters()
          "parameters": {
            "image": "artefact.skao.int/ska-sdp-realtime-receive-modules",
            "version": "3.3.0",
            "reception": {
              "num_channels": 128,
              "execution_block_id": EXECUTION_BLOCK_ID, # alternatives are schedblock filename or datamodel filename
              "sdp_config_backend": "etcd3",
              "sdp_config_host": os.environ["SDP_CONFIG_HOST"],
              "sdp_config_port": 2379,
            },
            "pvc": {
              "name": "shared"
            },
            "plasmaEnabled": True,
            "plasma_parameters": {
              "extraContainers": [
                  {
                    "name": "tmlite-server",
                    "image": "artefact.skao.int/ska-sdp-tmlite-server:0.3.0"
                  }
              ]
            }
          }
        },
        #{
        #  "pb_id": f"{PROCESSING_BLOCK_ID_REALTIME_SENDER}",
        #  "workflow": {"kind": "realtime": "name": "cbf-emulator", "version": "0.1.0"},
        #  "dependencies": [
        #    {"pb_id": f"{PROCESSING_BLOCK_ID_REALTIME_RECEIVER}", "kind": ["calibration"]}
        #  ]
        #},
        #{
        #  "pb_id": f"{PROCESSING_BLOCK_ID_BATCH}",
        #  "workflow": {"kind": "batch", "name": "test-dask", "version": "0.3.0"},
        #  "parameters": {},
        #  "dependencies": [
        #    {"pb_id": f"{PROCESSING_BLOCK_ID_REALTIME_RECEIVER}", "kind": ["calibration"]}
        #  ]
        #}
      ]
    }

In [308]:
from tango import DevState
import sys

tango_safe_release()
try:
    if d.state() == DevState.OFF:
        print(">> Device ON")
        d.On()
        wait_for_state(d, DevState.ON)
        wait_for_obs_state(d, d.obsState.EMPTY)

    print(">> Assigning Resources")
    config_eb = create_resources_config()
    # https://developer.skao.int/projects/ska-telmodel/en/latest/schemas/ska-sdp-assignres.html
    d.AssignResources(json.dumps(config_eb))
    wait_for_obs_state(d, d.obsState.IDLE)

    print(">> Get Receive Address")
    receiveAddresses = json.loads(d.receiveAddresses)
    print(receiveAddresses)

    host = receiveAddresses["science"]["host"][0][1]
    port = receiveAddresses["science"]["port"][0][1]
    receive_pb_config = create_receive_config()
    sender_pb_config = create_sender_config(host, port)
    
    print(">> Configure")
    # https://developer.skao.int/projects/ska-telmodel/en/latest/schemas/ska-sdp-configure.html
    d.Configure(json.dumps({"interface": "https://schema.skao.int/ska-sdp-configure/0.3", "scan_type": "science"}))
    wait_for_obs_state(d, d.obsState.READY)

    halt = True
    if halt:
        # TODO: halting here for pod testing
        print(">> Scan")
        d.Scan(json.dumps({"interface": "https://schema.skao.int/ska-sdp-scan/0.3", "scan_id": 0}))
        wait_for_obs_state(d, d.obsState.SCANNING)
        print("Now Receiving Scans")
    else:
        for scan in range(0,3):
            print(">> Scan")
            d.Scan(json.dumps({"interface": "https://schema.skao.int/ska-sdp-scan/0.3", "scan_id": scan}))
            wait_for_obs_state(d, d.obsState.SCANNING)
            sleep(10) # scanning for 10 seconds
            print(">> End Scan")
            d.EndScan()
            wait_for_obs_state(d, d.obsState.READY)

        print(">> End")
        d.End()
        wait_for_obs_state(d, d.obsState.IDLE)

        print(">> Releasing Resources")
        d.ReleaseResources()        
        wait_for_obs_state(d, d.obsState.EMPTY)

        print(">> Device OFF")
        d.Off()
        wait_for_state(d, DevState.OFF)

except Exception as e:
    # In case of failure, safely restore the device back to its original off state
    tango_safe_release()
    raise e

Tango Device is EMPTY
>> Device ON
Waiting for device state ON...
Waiting for device obs_state EMPTY...
>> Assigning Resources
Waiting for device obs_state IDLE...
>> Get Receive Address
{'calibration': {'host': [[0, 'proc-pb-notebook-20220916-29637-receive-0.receive.dp-shared-p.svc.cluster.local']], 'port': [[0, 41000, 1]]}, 'interface': 'https://schema.skao.int/ska-sdp-recvaddrs/0.3', 'science': {'host': [[0, 'proc-pb-notebook-20220916-29637-receive-0.receive.dp-shared-p.svc.cluster.local']], 'port': [[0, 41000, 1]]}}
>> Configure
Waiting for device obs_state READY...
>> Scan
Waiting for device obs_state SCANNING...
Now Receiving Scans


In [252]:
import datetime

def print_pods(namespace: str):
    pods = v1.list_namespaced_pod(namespace=namespace)
    print(f"pod_ip\t\tnamespace\tname\t\t\t\t\t\tphase")
    for i in pods.items:
        print(f"{str(i.status.pod_ip):10}\t{i.metadata.namespace}\t{i.metadata.name:20}\t{i.status.phase}")

def print_pod_events(name: str, namespace: str):
    field_selector = f"involvedObject.name={receive_pod_name}"
    stream = kubernetes.watch.Watch().stream(v1.list_namespaced_event, namespace=namespace, field_selector=field_selector, timeout_seconds=1)
    print(f"Events for {receive_pod_name}:")
    print("Type\tReason\t\t\tAge\t\tFrom\t\t\tMessage")
    for event in stream:
        print(
            f"{event['object'].type}\t"
            f"{event['object'].reason}\t"
            f"{event['object'].event_time}\t"
            #f"{datetime.datetime.now(datetime.timezone.utc)-event['object'].event_time}\t"
            f"{event['object'].reporting_component}\t"
            f"{event['object'].message}"
        )

In [309]:
#kubectl get pods -n {KUBE_NAMESPACE}
!kubectl get pods -n {KUBE_PROC_NAMESPACE}
#kubectl describe pod {receive_pod_name} -n {KUBE_PROC_NAMESPACE}

print_pods(KUBE_PROC_NAMESPACE)

receive_pod_name = f"proc-{config_eb['processing_blocks'][0]['pb_id']}-receive-0"
print_pod_events(receive_pod_name, KUBE_PROC_NAMESPACE)

# check logs
#print(v1.read_namespaced_pod_log(namespace=KUBE_NAMESPACE, name="ska-sdp-scripts-config--1-9rfbl"))
print(v1.read_namespaced_pod_log(namespace=KUBE_PROC_NAMESPACE, name=receive_pod_name))

No resources found in dp-yanda-callan-p namespace.
pod_ip		namespace	name						phase
Events for proc-pb-notebook-20220916-29637-receive-0:
Type	Reason			Age		From			Message


ApiException: (404)
Reason: Not Found
HTTP response headers: HTTPHeaderDict({'Audit-Id': '1bc54980-5c31-4e83-acf4-17d7d85c64b6', 'Cache-Control': 'no-cache, private', 'Content-Type': 'application/json', 'Date': 'Fri, 16 Sep 2022 07:58:00 GMT', 'Content-Length': '250'})
HTTP response body: {"kind":"Status","apiVersion":"v1","metadata":{},"status":"Failure","message":"pods \"proc-pb-notebook-20220916-29637-receive-0\" not found","reason":"NotFound","details":{"name":"proc-pb-notebook-20220916-29637-receive-0","kind":"pods"},"code":404}



In [310]:
tango_safe_release()

>> End Scan
Waiting for device obs_state READY...
>> End
Waiting for device obs_state IDLE...
>> Releasing Resources
Waiting for device obs_state EMPTY...
Tango Device is EMPTY


# Start Sender

In [ ]:
!sudo apt install wget
!pip install ska-sdp-cbf-emulator
!wget -nc https://gitlab.com/ska-telescope/sdp/ska-sdp-realtime-receive-core/-/raw/main/data/AA05LOW.ms.tar.gz
!tar xzvf AA05LOW.ms.tar.gz
!sudo apt install -y iputils-ping

In [94]:
import sys
import subprocess as sp

sp.Popen(f"ping {host}:{port}".split())

print(f"sending to {host}:{port}")
sender_pb_config = create_sender_config(host, port)
send_command = sender_pb_config["spec"]["template"]["spec"]["containers"][0]["command"]
sp.Popen(send_command)

sending to 10.10.4.51:9000


ping: 10.10.4.51:9000: Name or service not known


Traceback (most recent call last):
  File "/home/tango/.local/bin/emu-send", line 8, in <module>
    sys.exit(main())
  File "/home/tango/.local/lib/python3.7/site-packages/cbf_sdp/packetiser.py", line 201, in main
    packetise(config, args.measurement_set))
  File "/usr/lib/python3.7/asyncio/base_events.py", line 584, in run_until_complete
    return future.result()
  File "/home/tango/.local/lib/python3.7/site-packages/cbf_sdp/packetiser.py", line 51, in packetise
    ms = msutils.MeasurementSet.open(mspath)
  File "/home/tango/.local/lib/python3.7/site-packages/realtime/receive/core/msutils.py", line 514, in open
    return MeasurementSet(name, mode=mode)
  File "/home/tango/.local/lib/python3.7/site-packages/realtime/receive/core/msutils.py", line 551, in __init__
    self._open()
  File "/home/tango/.local/lib/python3.7/site-packages/realtime/receive/core/msutils.py", line 571, in _open
    self._t = tables.table(self._name, readonly=self._readonly, ack=False)
  File "/home/tango

In [22]:
for txn in config.txn():
    eb = txn.get_execution_block(config_eb['eb_id'])
    print(eb)
    receive_pb = txn.get_processing_block(eb["pb_realtime"][0])
    print(receive_pb.eb_id, receive_pb.pb_id)

{'current_scan_type': 'science', 'eb_id': 'eb-notebook-20220912-19606', 'pb_batch': ['pb-notebook-20220912-19607'], 'pb_realtime': ['pb-notebook-20220912-19606'], 'pb_receive_addresses': 'pb-notebook-20220912-19606', 'scan_id': 1, 'scan_types': [{'channels': [{'count': 10, 'freq_max': 368000000.0, 'freq_min': 350000000.0, 'link_map': [[0, 0], [200, 1], [744, 2], [944, 3]], 'start': 0, 'stride': 2}], 'coordinate_system': 'ICRS', 'dec': '-00:00:47.84', 'ra': '02:42:40.771', 'scan_type_id': 'science'}, {'channels': [{'count': 5, 'freq_max': 368000000.0, 'freq_min': 350000000.0, 'link_map': [[0, 0], [200, 1], [744, 2], [944, 3]], 'start': 0, 'stride': 2}], 'coordinate_system': 'ICRS', 'dec': '02:03:08.598', 'ra': '12:29:06.699', 'scan_type_id': 'calibration'}], 'status': 'ACTIVE', 'subarray_id': '02'}
eb-notebook-20220912-19606 pb-notebook-20220912-19606


In [24]:
!kubectl -n dp-shared get pods

The connection to the server localhost:8080 was refused - did you specify the right host or port?


In [ ]:
!ska-sdp --help

In [7]:
#!ska-sdp get /eb/eb-notebook-20220912-09050

In [125]:
# need to get processing block id of current scan
print(d.healthState)
print(d.get_logging_level())

healthState.OK
5


In [ ]:
# need to get processing_block logs

#print(PROCESSING_BLOCK_ID_BATCH)
#for txn in config.txn():
#     for pb in txn.list_processing_blocks():
#            print(pb)

In [40]:
# Turn Off Tango Device
tango_safe_off()

>> End Scan
Waiting for device obs_state READY...
>> End
Waiting for device obs_state IDLE...
>> Releasing Resources
Waiting for device obs_state EMPTY...
>> Device OFF
Waiting for device state OFF...
Tango Device is OFF


# Create Emulated Sender

In [ ]:
# Deploy Sender
for txn in config.txn():
    